In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import statsmodels.api as sm

import plotly.express as px
from plotly import graph_objs as go
from matplotlib import pyplot as plt

sys.path.append("../utils/")
from shot_chart_plots import plot_scatter, plot_scatter_single_df
# %matplotlib

plt.close("all")

In [ ]:
data_dir = "~/Documents/euroleague_api/notebooks/data"

In [ ]:
mid_range_x = (-245, 245)
mid_range_y = 422.5

# Shot Data

In [ ]:
shot_df = pd.read_csv(os.path.join(data_dir, "shot_data_2023.csv"))

In [ ]:
shot_df[['Season', 'Phase', 'Round', 'Gamecode', 'NUM_ANOT', 'TEAM', 'ID_PLAYER',
       'PLAYER', 'ID_ACTION', 'ACTION', 'POINTS', 'COORD_X', 'COORD_Y', 'ZONE', 'MINUTE',
       'CONSOLE', 'POINTS_A', 'POINTS_B', 'UTC']].head()

In [ ]:
shot_df["Round"].unique()

In [ ]:
shot_df["ZONE"].unique()

In [ ]:
# for zone in np.unique(shot_df.ZONE):
#     zone_df = shot_df[shot_df["ZONE"] == zone]
#     made = zone_df[zone_df['ID_ACTION'].isin(['2FGM', '3FGM'])]
#     missed = zone_df[zone_df['ID_ACTION'].isin(['2FGA', '3FGA'])]
#     plot_scatter(made, missed, title=zone)

In [ ]:
mid_range_zones = ["D", "E", "F", "G"]
non_mid_range_zones = [u for u in shot_df["ZONE"].unique() if u not in mid_range_zones + [" "]]
fg_zones = mid_range_zones + non_mid_range_zones

In [ ]:
# ensure the shot is for 2. There are some data irregularities where a 3PT shot is registered in the mid-range zones.
mid_range_mask = (
    shot_df["ZONE"].isin(mid_range_zones) &
    shot_df["ID_ACTION"].isin(['2FGM', '2FGA']) &
    (
        (shot_df["COORD_X"] < mid_range_x[0]) | (shot_df["COORD_X"] > mid_range_x[1]) | (shot_df["COORD_Y"] > mid_range_y)
    )
)

In [ ]:
midrange_df = shot_df[mid_range_mask]
fg_df =  shot_df[shot_df["ZONE"].isin(fg_zones)]

In [ ]:
fg_df["ID_ACTION"].unique()

In [ ]:
plot_scatter_single_df(midrange_df, title="Mid-range attempts until round 17 2023-2024")

In [ ]:
mid_range_shots_by_team_df = midrange_df.groupby("TEAM").count()["ID_ACTION"]
fg_by_team_df = fg_df.groupby("TEAM").count()["ID_ACTION"]
print((mid_range_shots_by_team_df.index == fg_by_team_df.index).all())
team_perc_midrange_df = (mid_range_shots_by_team_df / fg_by_team_df).sort_values()
team_perc_midrange_df

In [ ]:
n_fg_df = fg_df.groupby(["Gamecode", "TEAM"])[["Season"]].count()
n_midrange_df = midrange_df.groupby(["Gamecode", "TEAM"])[["Season"]].count()

In [ ]:
# some team register no midrange shots, count the total shots too and merge (outer) to ensure all gamecodes and teams are considered
n_midrange_fg_df = n_midrange_df.merge(n_fg_df, left_index=True, right_index=True, how="outer")
n_midrange_fg_df.columns = ["midrange", "fg"]
n_midrange_fg_df.fillna(0, inplace=True)
n_midrange_fg_df["midrange_perc"] = n_midrange_fg_df["midrange"] / n_midrange_fg_df["fg"]
n_midrange_fg_df.head()

In [ ]:
n_midrange_fg_df["t"] = ["A", "B"] * int(n_midrange_fg_df.shape[0] / 2 )
merged_reshaped_df = n_midrange_fg_df.reset_index().pivot(index="Gamecode", columns="t")[["TEAM", "midrange_perc"]]
merged_reshaped_df.columns = ["Team_A", "Team_B", "Midrange_perc_A", "Midrange_perc_B"]
merged_reshaped_df.head()

# Stadndings data

In [ ]:
standings_df = pd.read_csv(os.path.join(data_dir, "standings_2023_17.csv"))

# Merge Midrange % with Win %

In [ ]:
standings_percmidrange_df = standings_df[["club.code", "club.name", "winPercentage"]].merge(
    team_perc_midrange_df.reset_index(), left_on="club.code", right_on="TEAM")
standings_percmidrange_df["winPercentage"] = standings_percmidrange_df["winPercentage"].str.replace("%", "").astype(float)
standings_percmidrange_df.rename(columns={"ID_ACTION": "midRangePercentage"}, inplace=True)
standings_percmidrange_df["midRangePercentage"] = 100 * standings_percmidrange_df["midRangePercentage"]

In [ ]:
standings_percmidrange_df[["winPercentage", "midRangePercentage"]].corr()

In [ ]:
x_const = sm.add_constant(standings_percmidrange_df[["winPercentage"]], prepend=False)
# Fit and summarize OLS model
mod = sm.OLS(standings_percmidrange_df["midRangePercentage"], x_const)
res = mod.fit()

In [ ]:
# m, b = np.polyfit(x=standings_percmidrange_df["winPercentage"], y=standings_percmidrange_df["midRangePercentage"], deg=1)
m = res.params.winPercentage
b = res.params.const
x = np.arange(10, 100)
yfit = m * x + b

In [ ]:
res.summary()

In [ ]:
fig = px.scatter(standings_percmidrange_df, x="winPercentage", y="midRangePercentage", text="club.name")
fig.update_traces(textposition="bottom right")
fig.add_trace(go.Scatter(x=x, y=yfit, name="trend"))
fig.show()

# Game Report

In [ ]:
# Same as above, some teams shot no midrange, count the total 2PTs (i.e. layups too) and merge so that there no missing teams after the groupby
n_made_missed_midrange_df = midrange_df.groupby(["Gamecode", "TEAM", "ID_ACTION"])[["Season"]].count()
n_made_missed_2ps_df = shot_df[shot_df["ID_ACTION"].isin(['2FGM', '2FGA'])].groupby(["Gamecode", "TEAM", "ID_ACTION"])[["Season"]].count()

In [ ]:
n_made_missed_midrange_total2pt_df = n_made_missed_midrange_df.merge(n_made_missed_2ps_df, left_index=True, right_index=True, how="outer")
n_made_missed_midrange_total2pt_df.columns = ["midrage", "total"]
n_made_missed_midrange_total2pt_df.fillna(0, inplace=True)
n_made_missed_midrange_total2pt_df.head()

In [ ]:
game_report_df = pd.read_csv(os.path.join(data_dir, "game_report_2023.csv"))

In [ ]:
game_report_df.head()

In [ ]:
abbr_to_name_dict = game_report_df[["road.club.code", "road.club.name"]].drop_duplicates().set_index("road.club.code").to_dict()["road.club.name"]

In [ ]:
game_report_trunc_df = game_report_df[["Gamecode", "Round", "local.club.code", "road.club.code", "local.score", "road.score"]]

In [ ]:
fig = px.bar(100* team_perc_midrange_df.rename(index=abbr_to_name_dict))
fig.update_layout(
    yaxis={"title": "% Mid-range FGs"},
    showlegend=False
)

# Merge Game Report with Shot data analysis

In [ ]:
master_table = merged_reshaped_df.reset_index().merge(game_report_df, on="Gamecode")

In [ ]:
master_table["home_win"] = master_table["local.score"] > master_table["road.score"]

In [ ]:
master_table.head()

In [ ]:
# fix the order of team A and team B to match the order of local and road teams.
for i, row in master_table.iterrows():
    # break
    if (row[["Team_A", "Team_B"]].values == row[["local.club.code", "road.club.code"]].values).all():
        continue
    else:
        master_table.loc[i, ["Team_A", "Team_B"]] = row[["Team_B", "Team_A"]].values
        master_table.loc[i, ["Midrange_perc_A", "Midrange_perc_B"]] = row[["Midrange_perc_B", "Midrange_perc_A"]].values

In [ ]:
master_table["home_fewer_midrange_volume"] = master_table["Midrange_perc_A"] < master_table["Midrange_perc_B"]
master_table.head()

In [ ]:
n_wins_with_fewer_mid_range = (master_table["home_win"] & (master_table["home_fewer_midrange_volume"])).sum() + (~master_table["home_win"] & (~master_table["home_fewer_midrange_volume"])).sum()
n_games = master_table.shape[0]
ratio = n_wins_with_fewer_mid_range / n_games
print(ratio)

## Mid-range players

In [ ]:
mid_range_shots_by_player_df = midrange_df.groupby(["PLAYER", "ID_PLAYER", "TEAM"]).count()["ID_ACTION"]
fg_by_player_df = fg_df.groupby(["PLAYER", "ID_PLAYER", "TEAM"]).count()["ID_ACTION"]
player_perc_midrange_df = mid_range_shots_by_player_df.reset_index().merge(fg_by_player_df.reset_index(), on=["PLAYER", "ID_PLAYER", "TEAM"])
player_perc_midrange_df.columns = ["PLAYER", "ID_PLAYER", "TEAM", "mid-range", "fg"]
player_perc_midrange_df["midrange-perc"] = player_perc_midrange_df["mid-range"] / player_perc_midrange_df["fg"]
player_perc_midrange_df["TEAM"] = player_perc_midrange_df["TEAM"].replace(abbr_to_name_dict)
player_perc_midrange_df[player_perc_midrange_df["fg"] > 80].sort_values("midrange-perc", ascending=False).head(10)
# player_perc_midrange_df[player_perc_midrange_df["fg"] > 80].sort_values("midrange-perc", ascending=False).head(10)[["PLAYER", "TEAM", "midrange-perc"]]#.reset_index().to_markdown()